In [1]:
import pandas as pd

In [ ]:
# https://www.conab.gov.br/info-agro/safras/serie-historica-das-safras/itemlist/category/911-soja

In [3]:
!wget https://www.conab.gov.br/info-agro/safras/serie-historica-das-safras/item/download/47426_4ea19b29bec235ea569ae1c687d2f85e

--2023-05-23 21:40:38--  https://www.conab.gov.br/info-agro/safras/serie-historica-das-safras/item/download/47426_4ea19b29bec235ea569ae1c687d2f85e
Resolving www.conab.gov.br (www.conab.gov.br)... 160.238.29.73
Connecting to www.conab.gov.br (www.conab.gov.br)|160.238.29.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80384 (78K) [application/vnd.ms-excel]
Saving to: ‘47426_4ea19b29bec235ea569ae1c687d2f85e’

47426_4ea19b29bec23 100%[===================>]  78.50K  --.-KB/s    in 0.07s   

2023-05-23 21:40:39 (1.10 MB/s) - ‘47426_4ea19b29bec235ea569ae1c687d2f85e’ saved [80384/80384]



In [76]:
df = pd.read_excel("47426_4ea19b29bec235ea569ae1c687d2f85e.xls",
                   sheet_name="Produção",
                   skiprows=5,
                   nrows=35,
                   usecols="A:AV")

In [77]:
states = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT',
       'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO',
       'RR', 'SC', 'SP', 'SE', 'TO']

states_dict = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

In [78]:
df_2 = df[df['REGIÃO/UF'].str.len() == 2].reset_index()

In [79]:
cols = [
        'index', 'UF', '1976/77', '1977/78 ', '1978/79', '1979/80',
       '1980/81', '1981/82', '1982/83', '1983/84', '1984/85', '1985/86',
       '1986/87', '1987/88', '1988/89', '1989/90', '1990/91', '1991/92',
       '1992/93', '1993/94', '1994/95', '1995/96', '1996/97', '1997/98',
       '1998/99', '1999/00', '2000/01', '2001/02', '2002/03', '2003/04',
       '2004/05', '2005/06', '2006/07', '2007/08', '2008/09', '2009/10',
       '2010/11', '2011/12', '2012/13', '2013/14', '2014/15', '2015/16',
       '2016/17', '2017/18', '2018/19', '2019/20', '2020/21', '2021/22',
       '2022/23'
]

In [80]:
df_2.columns = cols

In [81]:
all_data = []
for state in states:
    c = list(df_2.query("UF == @state").columns)
    v = list(df_2.query("UF == @state").values[0])
    all_data.append(pd.DataFrame.from_dict({"safra": [x.split('/')[0] for x in c], "adm_1": states_dict[state], "producao": [x * 1000 for x in v]}))

In [82]:
df_producao = pd.concat(all_data)

In [83]:
df_producao[df_producao.safra.str.isnumeric()].to_parquet("output/producao_soja.parquet", version="1.0")

In [84]:
df_producao[df_producao.safra.str.isnumeric()].to_csv("output/producao_soja.csv", index=False)

In [85]:
df = pd.read_excel("47426_4ea19b29bec235ea569ae1c687d2f85e.xls",
                   sheet_name="Área",
                   skiprows=5,
                   nrows=35,
                   usecols="A:AV")

In [86]:
df_2 = df[df['REGIÃO/UF'].str.len() == 2].reset_index()

In [87]:
df_2.columns = cols

In [88]:
all_data = []
for state in states:
    c = list(df_2.query("UF == @state").columns)
    v = list(df_2.query("UF == @state").values[0])
    all_data.append(pd.DataFrame.from_dict({"safra": [x.split('/')[0] for x in c], "adm_1": states_dict[state], "area": [x * 1000 for x in v]}))

In [89]:
df_area = pd.concat(all_data)

In [90]:
df_area[df_area.safra.str.isnumeric()].to_parquet("output/area_soja.parquet", version="1.0")

In [91]:
df_tudo = pd.concat([df_producao.set_index(['safra', 'adm_1']), df_area.set_index(['safra', 'adm_1'])], axis=1).reset_index().fillna(0)

In [93]:
df_tudo[df_tudo.safra.str.isnumeric()].to_parquet("output/area_e_producao_soja.parquet", version="1.0")